# Mean results plot
Simple notebook to plot figures for the paper.

Authors:
Edoardo Daniele Cannas - edoardodaniele.cannas@polimi.it  
Sriram Baireddy - sbairedd@purdue.edu  
Paolo Bestagini - paolo.bestagini@polimi.it  
Stefano Tubaro - stefano.tubaro@polimi.it  
Edward J. Delp - ace@purdue.edu  

## Libraries import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import cv2
from sklearn.metrics import roc_curve, auc
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Times New Roman']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

## Helpers

In [ ]:
from sklearn.preprocessing import RobustScaler, QuantileTransformer

def quantile_scaling(img: np.ndarray, dist: str) -> np.ndarray:
	"""
	QuantileTransformer: maps the image range to a specific stastical distribution
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:param distribution: str, distribution for mapping the image (either 'uniform' or 'normal')
	:return: np.ndarray, normalized numpy array
	"""
	return QuantileTransformer(output_distribution=dist).fit_transform(img.reshape(-1, 1)).reshape(img.shape)


def rename_sat(product: str) -> str:
    if product[0] == 'L':
        sat_name = 'Landsat8'
    else:
        sat_name = product.split('_')[0]
        if sat_name in ['AMAZONIA', 'CBERS']:
            sat_name += f'{product.split("_")[1]}_{product.split("_")[2]}'
    return sat_name


def compute_model_sign(train_tag: str) -> str:
    # Create a tag for the visualization
    model_sign = train_tag.split('/')[-3]
    return model_sign
        

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

## Load the results

### Histogram Equalization Generated Dataset (HEGD)

In [ ]:
# Look for all models used
results_dir = '../results/hegd'  # PLACE YOUR RESULTS DIRECTORY HERE!
all_dfs_paths = glob.glob(os.path.join(results_dir, '**', 'results_df_mean_fingerprints.pkl'), recursive=True)

# Load the Dataframe for each model and put it in a bigger one
all_results_df = []
for path in all_dfs_paths:cn
    # Load results
    df = pd.read_pickle(path)
    # Create a tag for the visualization
    model_sign = compute_model_sign(path)
    # Save in the bigger DataFrame
    df = pd.concat({model_sign: df}, names=['Model_name', 'Operation', 'Img_idx'])
    all_results_df.append(df)

# Concatenate everything
all_results_df = pd.concat(all_results_df)

# Compute the mean across operations
he_mean_results = all_results_df.groupby(['Model_name', 'Operation']).mean()[['AUC']]


### Standard Generated Dataset (SGD)

In [ ]:
# Look for all models used
results_dir = '../results/sgd'  # PLACE YOUR RESULTS DIRECTORY HERE!
all_dfs_paths = glob.glob(os.path.join(results_dir, '**', 'results_df_mean_fingerprints.pkl'), recursive=True)

# Load the Dataframe for each model and put it in a bigger one
all_results_df = []
for path in all_dfs_paths:
    # Load results
    df = pd.read_pickle(path)
    # Create a tag for the visualization
    model_sign = compute_model_sign(path)
    # Save in the bigger DataFrame
    df = pd.concat({model_sign: df}, names=['Model_name', 'Operation', 'Img_idx'])
    all_results_df.append(df)

# Concatenate everything
all_results_df = pd.concat(all_results_df)

# Compute the mean across operations
max_abs_mean_results = all_results_df.groupby(['Model_name', 'Operation']).mean()[['AUC']]


### Join together the Dataframes

In [ ]:
all_results = max_abs_mean_results.join(he_mean_results, rsuffix=' HEG', lsuffix=' SG')
all_results

## Plot the results
We are going to have a separate bar plot for the SGD and the HEGD datasets.

### Re-organize the Dataframe

In [ ]:
# Select the models to plot
plot_df = all_results.loc[['MinPMax_95', 'MinPMax_99', 'MaxScaling', 'HistogramEqualization']]
# Rename the columns
plot_df = plot_df.rename({'MinPMax_95': r'MinPMax $95^{th}$ percentile',
                          'MinPMax_99': r'MinPMax $99^{th}$ percentile',
                          'MaxScaling': r'MaxScaling',
                          'HistogramEqualization': 'HistogramEqualization'})
# Rename the index
plot_df = plot_df.rename_axis(index={'Model_name': 'Preprocessing strategy'})
# Rename the operations
operations_mapping = {'Affine': 'Rotation + resize', 'AverageBlur': 'Average blur', 'GaussianBlur': 'Gaussian Blur',
                      'Identity': 'No editing', 'LogContrast': 'Log contrast', 'MotionBlur': 'Motion blur',
                      'PerspectiveTransform': 'Prospective affine', 'PiecewiseAffine': 'Piecewise affine', 
                      'SigmoidContrast': 'Sigmoid contrast'}
plot_df.index = plot_df.index.set_levels([plot_df.index.levels[0], plot_df.index.levels[1].map(lambda x: operations_mapping[x.replace('Unnamed', '')])])
plot_df = plot_df.sort_index()
plot_df


### Plot the SGD results

In [ ]:
# Plot params
plt.rcParams['font.family'] = 'Times New Roman'
fontsize=20
ax = plot_df['AUC SG'].swaplevel(0, 1).unstack(level=1).plot.barh(legend=True, 
                                                                 xlim=[0, 0.95], 
                                                                 xticks=np.linspace(0, 1.0, 11), 
                                                                 grid=True, figsize=(7, 5))

# Adjust the layout
ax.tick_params(axis='both', labelsize=fontsize-5)
ax.set_ylabel('Editing operation', fontsize=fontsize)
ax.set_xlabel('AUC value', fontsize=fontsize)
ax.axvline(x=plot_df['AUC SG'].mean(), color='black', linestyle='--', label='Mean performance')
legend = ax.legend(loc='upper left', prop={'size': fontsize-5})
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(1)


### Plot the HEGD results

In [ ]:
# Plot params
plt.rcParams['font.family'] = 'Times New Roman'
fontsize=20
ax = plot_df['AUC HEG'].swaplevel(0, 1).unstack(level=1).plot.barh(legend=True, 
                                                                 xlim=[0, 0.95], 
                                                                 xticks=np.linspace(0, 1.0, 11), 
                                                                 grid=True, figsize=(7, 5))

# Adjust the layout
ax.tick_params(axis='both', labelsize=fontsize-5)
ax.set_ylabel('Editing operation', fontsize=fontsize)
ax.set_xlabel('AUC value', fontsize=fontsize)
ax.axvline(x=plot_df['AUC HEG'].mean(), color='black', linestyle='--', label='Mean performance')
legend = ax.legend(loc='upper left', prop={'size': fontsize-5})
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(1)
